# FizzBuzz in TensorFlow

## Dataset

First, let's try to get the dataset for our problem. Here is a simple one-liner to prepare the training data

In [ ]:
def get_training_data(min_range:int = 1, max_range:int = 100):
    return {x: 'fizzbuzz' if x % 15==0 else "buzz" if x%5==0 else "fizz" if x%3 == 0 else x for x in range(min_range,max_range)}

In [ ]:
data = get_training_data(101, 1024)

## Machine Learning

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

In [ ]:
labels = ['Number', 'Fizz','Buzz','FizzBuzz']

def convert_to_vector(num):
    n = num
    out = []
    while n > 0:
        out.append(n%10)
        n //= 10
    
    return out[::-1]

def convert_to_int(label):
    if label in labels:
        return labels.index(label)
    else:
        return 0

def fizz_buzz_pred(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]



In [ ]:
X = list(map(convert_to_vector, data.keys()))
y = list(map(convert_to_int, data.values()))




X_vector = pad_sequences(np.array(X, dtype=object))
y_vector = np.array(y)


X_train, X_test, y_train, y_test = train_test_split(X_vector,y_vector, test_size=0.2,shuffle=True)

## Training

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *

In [ ]:

embed_size = 3
sequence_length =4
model = tf.keras.Sequential()
model.add(Input(shape=(sequence_length,)))

model.add(Embedding(10, embed_size, trainable=True ))

model.add(Conv1D(64, kernel_size=2, padding='same',kernel_initializer=tf.keras.initializers.HeNormal(), activation='relu'))
model.add(MaxPool1D(pool_size=2, strides=1, padding='valid'))
model.add(GlobalAveragePooling1D())
model.add(Dense(32, activation='relu',kernel_initializer=tf.keras.initializers.HeNormal()))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['sparse_categorical_accuracy'])

2021-12-12 20:21:12.965335: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-12 20:21:12.965875: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-12 20:21:12.969578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-12 20:21:12.970094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-12 20:21:12.970582: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [ ]:
ACCURACY_THRESHOLD = 0.99
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_sparse_categorical_accuracy') > ACCURACY_THRESHOLD):
            print('Reached %2.2f%% accuracy, on the epoch %d so stopping training' %(ACCURACY_THRESHOLD, epoch))
            self.model.stop_training=True

In [ ]:
cb = myCallback()

#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=4000, batch_size=64)

In [ ]:
model.evaluate(X_test, y_test)

6/6 [==============================] - 0s 2ms/step - loss: 0.8100 - sparse_categorical_accuracy: 1.0000


[0.8100208044052124, 1.0]

## Evaluation

In [ ]:
inputs = np.array(list(range(1,101)))
input_vector = list(map(convert_to_vector, inputs))
input_vector = pad_sequences(np.array(input_vector, dtype=object), maxlen=sequence_length)

predictions = model(input_vector).numpy()


def convert_fb(num):
    if num % 15 == 0:
        return "fizzbuzz"
    
    elif num % 3 == 0:
        return "fizz"
    
    elif num % 5 == 0:
        return "buzz"
    
    else:
        return str(num)
    
acc = 0
total = 0
for i,inp in enumerate(inputs):
    pred = np.argmax(predictions[i])
    print(inp, fizz_buzz_pred(inp, pred), convert_fb(inp), predictions[i][pred])
    
    acc += 1 if fizz_buzz_pred(inp, pred) == convert_fb(inp) else 0
    total += 1
    
print(f'Accuracy = {acc*1.0 / total}')
    
    

1 1 1 0.43005535
2 2 2 0.41603026
3 3 fizz 0.4052864
4 4 4 0.4260611
5 5 buzz 0.42870843
6 6 fizz 0.40447542
7 7 7 0.43411493
8 8 8 0.4061676
9 9 fizz 0.41737923
10 10 buzz 0.4487515
11 11 11 0.43635663
12 12 fizz 0.4291185
13 13 13 0.4609857
14 14 14 0.43829203
15 15 fizzbuzz 0.4400162
16 16 16 0.44508687
17 17 17 0.4766092
18 18 fizz 0.45149398
19 19 19 0.47132048
20 20 buzz 0.42525625
21 21 fizz 0.42843023
22 22 22 0.4095408
23 23 23 0.45085606
24 24 fizz 0.42478704
25 25 buzz 0.42848253
26 26 26 0.42830172
27 27 fizz 0.4685334
28 28 28 0.442295
29 29 29 0.4575329
30 30 fizzbuzz 0.41230696
31 31 31 0.44853008
32 32 32 0.44230172
33 33 fizz 0.40443528
34 34 34 0.4501929
35 35 buzz 0.45186588
36 36 fizz 0.4201126
37 37 37 0.42734745
38 38 38 0.40468997
39 39 fizz 0.41574597
40 40 buzz 0.44353008
41 41 41 0.42822433
42 42 fizz 0.42284435
43 43 43 0.46070608
44 44 44 0.4284737
45 45 fizzbuzz 0.43036503
46 46 46 0.43489158
47 47 47 0.47919744
48 48 fizz 0.45178995
49 49 49 0.47126514
50 